In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import pandas as pd
import numpy as np
import datetime as dt
import importlib
import feature_functions
from feature_functions import *
import pandas as pd
from pandasql import sqldf

In [300]:
importlib.reload(feature_functions)
from feature_functions import *

In [301]:
prices = loaddata_4h('AUDUSD_Candlestick_4_Hour_BID_03.08.2003-04.09.2021_EEF_GMT.csv')

In [302]:
rsiKey = [13]
tdiKey = [13]
dictList = []
colFeat = []
keyList = []
rsiDict   = rsi(prices, rsiKey)
tdiDict   = tdi1(prices, tdiKey)
# dictList.append(rsiDict.df); colFeat.append('rsi'); keyList.append(rsiKey) 
dictList.append(tdiDict.df); colFeat.append('tdi'); keyList.append(tdiKey) 

In [303]:
# prices.dropna(inplace=True)
masterFrame = pd.DataFrame(index=prices.index)
masterFrame[['id','date','hour','open','high','low','close','volume']] = prices[['id','date','hour','open','high','low','close','volume']]   
masterFrame['weekday'] = prices.date.apply(lambda x: x.isoweekday())

In [304]:
for i in range(0,len(dictList)):#iteracja po funkcjach
    for j in keyList[i]: #iteracja po okresach
        for k in list(dictList[i][j]): #iteracja po kolumnach danej funkcji w okresie
            colID = colFeat[i] + str(j) + k
            masterFrame[colID] = dictList[i][j][k]

In [305]:
# masterFrame.to_csv(sep=';',path_or_buf='../Data/AUDUSD_rdi.csv',date_format="%Y-%m-%d",index = False,na_rep='')

In [306]:
masterFrame.iloc[-30:,:]

,id,date,hour,open,high,low,close,volume,weekday,tdi13rsi1,...,tdi13red2,tdi13green2,tdi13green2_red2,tdi13green_red_change,tdi13red_slope,tdi13green_slope,tdi13green_red_slope_change,tdi13habarsize1,tdi13hacolor1,tdi13barnumber1
28095,28095,2021-08-29 21:00:00,21,0.73029,0.73163,0.73029,0.73125,3706.7701,7,68.356734,...,56.551047,69.587916,13.036869,-4.013459,2.782277,-1.231182,-4.013459,-0.000892,-1,1
28096,28096,2021-08-30 01:00:00,1,0.73125,0.73181,0.72960,0.73034,10735.4001,1,69.056077,...,59.433230,68.706406,9.273176,-2.984544,3.334216,0.349672,-2.984544,0.000433,1,1
28097,28097,2021-08-30 05:00:00,5,0.73034,0.73063,0.72840,0.72958,11589.8899,1,65.115203,...,62.204463,67.085640,4.881177,-3.859093,1.888656,-1.970437,-3.859093,-0.000433,-1,3
28098,28098,2021-08-30 09:00:00,9,0.72959,0.73044,0.72871,0.72933,10120.8500,1,61.918197,...,63.636404,63.516700,-0.119704,-2.500553,0.902050,-1.598503,-2.500553,-0.000259,-1,4
28099,28099,2021-08-30 13:00:00,13,0.72935,0.73048,0.72874,0.72932,12745.0098,1,60.853479,...,64.386352,61.385838,-3.000513,-1.427760,0.895401,-0.532359,-1.427760,-0.000139,-1,5
28100,28100,2021-08-30 17:00:00,17,0.72932,0.72989,0.72913,0.72951,3591.0200,1,60.808168,...,65.275279,60.830823,-4.444456,1.407477,-1.430133,-0.022655,1.407477,-0.000084,-1,6
28101,28101,2021-08-30 21:00:00,21,0.72945,0.73023,0.72878,0.72974,6278.0302,1,61.399758,...,63.929659,61.103963,-2.825696,1.289649,-0.993854,0.295795,1.289649,0.000011,1,2
28102,28102,2021-08-31 01:00:00,1,0.72975,0.73107,0.72876,0.73104,10512.7800,2,62.149029,...,63.042844,61.774393,-1.268451,1.361357,-0.986721,0.374635,1.361357,0.000178,1,3
28103,28103,2021-08-31 05:00:00,5,0.73105,0.73409,0.73103,0.73271,16148.8003,2,66.169978,...,62.630545,64.159503,1.528959,1.859792,0.150682,2.010475,1.859792,0.000775,1,4
28104,28104,2021-08-31 09:00:00,9,0.73269,0.73357,0.73176,0.73236,16518.6604,2,70.527187,...,63.403685,68.348583,4.944897,0.948749,1.229856,2.178604,0.948749,0.000640,1,5


In [307]:
masterFrame = opentrades(masterFrame,5)

In [308]:
masterFrame.tail(3)

,id,date,hour,open,high,low,close,volume,weekday,tdi13rsi1,...,tdi13green_red_change,tdi13red_slope,tdi13green_slope,tdi13green_red_slope_change,tdi13habarsize1,tdi13hacolor1,tdi13barnumber1,tradetype,openprice,openindex
28122,28122,2021-09-03 09:00:00,9,0.74327,0.74775,0.74287,0.74573,23763.06,5,81.879704,...,1.311578,0.964294,2.275872,1.311578,0.001295,1,8,-1,0.74327,28122
28123,28123,2021-09-03 13:00:00,13,0.74575,0.74667,0.74313,0.74524,28589.83,5,84.943538,...,0.303793,1.228124,1.531917,0.303793,0.001048,1,9,-1,0.74575,28123
28124,28124,2021-09-03 17:00:00,17,0.74523,0.74636,0.74470,0.74470,4633.30,5,81.965090,...,-2.128993,0.639768,-1.489224,-2.128993,-0.000116,-1,1,-1,0.74523,28124


In [309]:
importlib.reload(feature_functions)
from feature_functions import *


In [310]:
masterFrame = closetrades(masterFrame,9)

In [311]:
masterFrame.tail(10)

,id,date,hour,open,high,low,close,volume,weekday,tdi13rsi1,...,tdi13green_slope,tdi13green_red_slope_change,tdi13habarsize1,tdi13hacolor1,tdi13barnumber1,tradetype,openprice,openindex,closeindex,closeprice
28112,28112,2021-09-01 17:00:00,17,0.73752,0.73772,0.73640,0.73654,6504.58,3,76.916330,...,2.277671,0.102184,0.001136,1,4,-1,0.73752,28112,28116,0.73883
28113,28113,2021-09-01 21:00:00,21,0.73661,0.73733,0.73580,0.73615,5733.51,3,71.484984,...,-2.715673,-3.951325,-0.000428,-1,1,-1,0.73661,28113,28116,0.73883
28114,28114,2021-09-02 01:00:00,1,0.73615,0.73673,0.73554,0.73672,10383.87,4,69.414065,...,-1.035460,-2.295160,-0.000276,-1,2,-1,0.73615,28114,28116,0.73883
28115,28115,2021-09-02 05:00:00,5,0.73670,0.73890,0.73662,0.73883,16205.47,4,70.755486,...,0.670710,-0.691932,0.000340,1,1,1,0.73670,28115,28116,0.73883
28116,28116,2021-09-02 09:00:00,9,0.73883,0.73963,0.73814,0.73950,12330.79,4,75.129649,...,2.187081,1.615728,0.001066,1,2,-1,0.73883,28116,28122,0.74327
28117,28117,2021-09-02 13:00:00,13,0.73953,0.74091,0.73897,0.74014,20509.01,4,76.346668,...,0.608509,0.039127,0.000435,1,3,-1,0.73953,28117,28122,0.74327
28118,28118,2021-09-02 17:00:00,17,0.74014,0.74035,0.73969,0.73987,8243.98,4,77.486711,...,0.570022,0.488539,0.000279,1,4,-1,0.74014,28118,28122,0.74327
28119,28119,2021-09-02 21:00:00,21,0.73987,0.74050,0.73949,0.74037,3983.92,4,75.816623,...,-0.835044,-1.453850,-0.000139,-1,1,-1,0.73987,28119,28122,0.74327
28120,28120,2021-09-03 01:00:00,1,0.74039,0.74083,0.73952,0.74062,10170.65,5,76.818959,...,0.501168,-0.556674,0.000301,1,6,-1,0.74039,28120,28122,0.74327
28121,28121,2021-09-03 05:00:00,5,0.74062,0.74373,0.74043,0.74326,14345.26,5,77.327961,...,0.254501,-0.684424,0.000197,1,7,1,0.74062,28121,28122,0.74327
